In [5]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path

from devtools import debug  # noqa: E402

# fmt:off
_ = [sys.path.append(str(path)) for path in [Path.cwd(), Path.cwd().parent, Path.cwd().parent / "python"] if str(path) not in sys.path] #fmt:on
import re

from langchain_community.document_loaders import WebBaseLoader

from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt

URL = "https://www.agroparistech.fr/formations-ingenieur/mention-biodiversite-ecologie-evolution-montpellier-parcours-m1-biodiversite-vegetale-gestion-ecosystemes-tropicaux-montpellier"

loader = WebBaseLoader(URL)
doc = loader.load()
page = doc[0].page_content.strip()
page_cleaned = re.sub(r'[\t ]+', ' ', page)
page_cleaned = re.sub(r'\n{3,}', '\n\n', page_cleaned)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/tcl/prj/genai-blueprint/python/ai_core/prompts.py:115 Llama2Format
    PromptFormatter: <class 'python.ai_core.prompts.PromptFormatter'> (ModelMetaclass)


In [6]:
# cSpell:disable

from devtools import debug
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class Parcours(BaseModel):
    metiers: list[str] = Field(
        description="liste de metiers auquels peut conduire la formation"
    )
    domaines: str = Field(description="domaines auquels s'applique cette formation")


system = """
    Extraire les metiers et les domaines d'une fiche de description de formation récupérée sur internet.  
    Repondre seulement avec les informations disponibles dans la description.
    \n
    """
user = """
    fiche de desctiption de formation:
    --- 
    {fiche}
    --- 
    {format_instructions}"""

parser = PydanticOutputParser(pydantic_object=Parcours)

prompt = def_prompt(system=system, user=user).partial(
    format_instructions=parser.get_format_instructions()
)

# LLM = "llama3_70_groq"
LLM = "llama3_8_deepinfra"
LLM = "mixtral_7x8_deepinfra"
LLM = "llama3_70_deepinfra"
LLM = "llama3_70_groq"
# LLM = "mixtral_7x8_groq"
# LLM = "mixtral_7x8_deepinfra"

# set_debug(True)
# set_verbose(True)

llm = get_llm(llm_id=LLM)
chain = prompt | llm | parser
summaries = chain.batch([{"fiche": page_cleaned}], {"max_concurrency": 5})

debug(summaries)

2024-05-30 11:39:24.392 | INFO     | python.ai_core.llm:set_cache:334 - cache: sqlite


/tmp/ipykernel_503551/2517771525.py:48 <module>
    summaries: [
        Parcours(
            metiers=[
                'Rechercheur',
                'Enseignant-chercheur',
                'Ingénieur',
                'Gestionnaire de la biodiversité',
                'Conservateur de la nature',
                'Gestionnaire des écosystèmes',
                'Spécialiste de la gestion des écosystèmes',
                'Chercheur en écologie',
                'Chercheur en biologie',
                "Chercheur en sciences de l'environnement",
                'Expert en biodiversité',
                'Expert en gestion des écosystèmes',
            ],
            domaines=(
                'Biodiversité, écologie, évolution, gestion des écosystèmes, environnement, conservation, sciences de '
                "la vie, sciences de l'environnement"
            ),
        ),
    ] (list) len=1


[Parcours(metiers=['Rechercheur', 'Enseignant-chercheur', 'Ingénieur', 'Gestionnaire de la biodiversité', 'Conservateur de la nature', 'Gestionnaire des écosystèmes', 'Spécialiste de la gestion des écosystèmes', 'Chercheur en écologie', 'Chercheur en biologie', "Chercheur en sciences de l'environnement", 'Expert en biodiversité', 'Expert en gestion des écosystèmes'], domaines="Biodiversité, écologie, évolution, gestion des écosystèmes, environnement, conservation, sciences de la vie, sciences de l'environnement")]

In [ ]:
# cSpell:disable

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class Parcours(BaseModel):
    metiers: list[str] = Field(
        description="liste de metiers auquels peut conduire la formation"
    )
    domaines: list[str] = Field(
        description="domaines auquels s'applique cette formation"
    )


class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")


system = """
    Extraire d'une fiche de desctiption de formation récupérée sur internet.  
    Repondre seulement avec les informations disponibles dans la description.
    \n
    """
user = """
    fiche de desctiption de formation:
    --- 
    {fiche} """

LLM = "llama3_70_groq"
llm = get_llm(llm_id=LLM)
structured_llm = llm.with_structured_output(Parcours)
chain = prompt | llm
res = chain.batch([{"fiche": page_cleaned}], {"max_concurrency": 5})
debug(res)

In [ ]:
class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")


system = """
    Tell me a joke about a topic. respond in JSON
    """
user = """
    format : respond in json.
    topic : {topic}
     """

LLM = "llama3_70_groq"
# LLM = "gpt_35_openai"
# LLM = "llama3_70_deepinfra"

# LLM = "llama3_8_deepinfra"

parser = PydanticOutputParser(pydantic_object=Joke)

prompt = def_prompt(user=user).partial(
    format_instructions=parser.get_format_instructions()
)
llm = get_llm(llm_id=LLM)
structured_llm = llm.with_structured_output(Joke)

prompt = def_prompt(user=user)

chain = prompt | structured_llm


res = chain.invoke({"topic": "german"})
debug(res)

In [7]:
from python.ai_core.loaders import load_docs_from_jsonl

REPO = Path("/mnt/c/Users/a184094/OneDrive - Eviden/_En cours/mon_master/")
FILES = REPO / "synthesis.json"
docs = list(load_docs_from_jsonl(FILES))
urls = [doc.metadata.get("lien_fiche") for doc in docs]
urls = {url for url in urls if url and url.count("/") > 5}

In [8]:
len(urls)

1959

In [9]:
import re

import enchant
from unidecode import unidecode

french_dict = enchant.Dict("fr")

english_dict = enchant.Dict("en")

In [10]:
import re

import enchant
from langchain.schema import Document
from unidecode import unidecode

french_dict = enchant.Dict("fr")
english_dict = enchant.Dict("en")


def find_acronyms(docs: list[Document]):
    acronyms = set()
    candidates = set()
    for doc in docs:
        regexp = r"(?<!\()\b[A-Z]{2,}\b(?!\))"  # r"\b[A-Z]{2,}\b"

        candidates.update(re.findall(regexp, doc.page_content))
        # debug(candidates)
    for word in candidates:
        if not french_dict.check(word) and not english_dict.check(word):
            suggested = [unidecode(w).lower() for w in french_dict.suggest(word)]
            if unidecode(word).lower() not in suggested:
                acronyms.add(word)
    return acronyms


a = find_acronyms(docs)
a

{'ACI',
 'ACNCC',
 'ACQ',
 'ACSYON',
 'ADIR',
 'AEFE',
 'AEQA',
 'AETBV',
 'AGEST',
 'AGRO',
 'AMO',
 'ANDALUS',
 'APA',
 'APAS',
 'APRI',
 'APS',
 'APSA',
 'AQMA',
 'ARCHEOMETRIE',
 'ARPAC',
 'ARTICC',
 'ASFORED',
 'BAPT',
 'BGB',
 'BIOGET',
 'BIOREF',
 'BME',
 'BSE',
 'BTP',
 'CAFEP',
 'CAFERUIS',
 'CAPEPS',
 'CAPESA',
 'CAPLP',
 'CAPPEI',
 'CCA',
 'CCMO',
 'CDMAT',
 'CDORG',
 'CDPI',
 'CGAO',
 'CGPI',
 'CIED',
 'CIEF',
 'CIMPN',
 'CLCC',
 'CLG',
 'CLILLE',
 'CMT',
 'COPEDA',
 'CPE',
 'CPTS',
 'CRM',
 'CRO',
 'CRYPTIS',
 'CTM',
 'CYBER',
 'CYBERSECURITE',
 'CYBERUS',
 'CYU',
 'DCE',
 'DCG',
 'DEAA',
 'DEDI',
 'DEVOPS',
 'DIFLES',
 'DJCE',
 'DNL',
 'DPM',
 'DSCG',
 'DSEM',
 'DSN',
 'DSP',
 'DUMI',
 'DYNAMEID',
 'EACM',
 'EAD',
 'ECODEVA',
 'ECOSAFE',
 'EDSB',
 'EDUNIVERSAL',
 'EEA',
 'EEAE',
 'EFMD',
 'EGEN',
 'EIFFALE',
 'EIPHI',
 'ELECTROMOBILITE',
 'EMC',
 'EMCC',
 'EMI',
 'EMIMEP',
 'EMJM',
 'EMN',
 'EMSAR',
 'ENJEUFOR',
 'ENR',
 'EPDIS',
 'EPE',
 'EPHE',
 'EPIED',
 'EPIGENBIO',
 

In [11]:
len(a)

314

In [ ]:
french_dict.suggest("economie")

In [ ]:
broker.describe()